In [134]:
import csv
from collections import namedtuple


def get_dialect(file):
    with open(file) as f:
        return csv.Sniffer().sniff(f.read(1000))


class CSVParser:
    def __init__(self, file_path):
        self._file = open(file_path)
        self.reader = csv.reader(
            self._file, get_dialect(file_path)
        )
    
    def __enter__(self):
        headers = map(str.lower, next(self.reader))
        self.ntuple = namedtuple("Generic", headers)
        return self
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        self._file.close()
        return False
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._file.closed:
            raise StopIteration
        
        next_line = next(self.reader)
        return self.ntuple(*next_line)

In [141]:
with CSVParser("personal_info.csv") as file:
    print(next(file))

Generic(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')


In [142]:
next(file)

StopIteration: 

In [137]:
from contextlib import contextmanager

In [174]:
@contextmanager
def csv_parse(file):
    _f = open(file)
    reader = csv.reader(_f, csv.Sniffer().sniff(_f.read(1000)))
    _f.seek(0)
    headers = map(str.lower, next(reader))
    ntuple = namedtuple("Generic", headers)
    
    try:
        yield (ntuple(*l) for l in reader)
    finally:
        _f.close()

In [175]:
with csv_parse("personal_info.csv") as file:
    for obj in file:
        pass
next(file)

StopIteration: 

In [151]:
def parse_data_iter(data_iter, nt):
    for row in data_iter:
        yield nt(*row)

In [152]:
@contextmanager
def parsed_data(f_name):
    f = open(f_name, 'r')
    try:
        reader = csv.reader(f, get_dialect(f_name))
        headers = map(str.lower, next(reader))
        nt = namedtuple("Data", headers)
        yield parse_data_iter(reader, nt)
    finally:
        f.close()

In [153]:
with parsed_data("personal_info.csv") as file:
    for obj in file:
        pass
next(file)

StopIteration: 

In [167]:
def test_speed(fn):
    with fn("personal_info.csv") as file:
        for obj in file:
            pass

In [168]:
from timeit import timeit

In [169]:
timeit("test_speed(csv_parse)", number=10*4, globals=globals())

0.1390869400001975